In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import input_data
import os

In [2]:
data = input_data.read_data_sets('data/fashion',one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [3]:
print('Training set(image) shape:{}'.format(data.train.images.shape))
print('Training set(label) shape:{}'.format(data.train.labels.shape))
print('Test set(image) shape:{}'.format(data.test.images.shape))
print('Test set(label) shape:{}'.format(data.test.labels.shape))

Training set(image) shape:(55000, 784)
Training set(label) shape:(55000, 10)
Test set(image) shape:(10000, 784)
Test set(label) shape:(10000, 10)


In [4]:
label_dict ={
    0:'T-shrit/top',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8:'Bag',
    9:'Ankle boot'
}

print(data.train.labels)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [5]:
#Reshaping Data to 28 X 28
train_X = data.train.images.reshape(-1,28,28,1)
test_X = data.test.images.reshape(-1,28,28,1)
train_X.shape,test_X.shape

((55000, 28, 28, 1), (10000, 28, 28, 1))

In [6]:
traing_iters = 200
learning_rate = 0.001
batch_size = 128
n_input = 28
n_classes = 10

In [7]:
tf.compat.v1.disable_eager_execution()
x = tf.compat.v1.placeholder('float',[None,28,28,1])
y= tf.compat.v1.placeholder('float',[None,n_classes])

In [8]:
def conv2d(x,W,b,strides=1):
    #conv2d wrapper with bias and relu
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

In [10]:
weights = {
    'wc1':tf.compat.v1.get_variable('W0',shape=(3,3,1,32),initializer=tf.keras.initializers.GlorotNormal()),
    'wc2':tf.compat.v1.get_variable('W1',shape=(3,3,32,64),initializer=tf.keras.initializers.GlorotNormal()),
    'wc3':tf.compat.v1.get_variable('W2',shape=(3,3,64,128),initializer=tf.keras.initializers.GlorotNormal()),
    'wd1':tf.compat.v1.get_variable('W3',shape=(4*4*128,128),initializer=tf.keras.initializers.GlorotNormal()),
    'out':tf.compat.v1.get_variable('W4',shape=(128,n_classes),initializer=tf.keras.initializers.GlorotNormal())

}

biases = {
    'bc1':tf.compat.v1.get_variable('B0',shape=(32),initializer = tf.keras.initializers.GlorotNormal()),
    'bc2':tf.compat.v1.get_variable('B1',shape=(64),initializer = tf.keras.initializers.GlorotNormal()),
    'bc3':tf.compat.v1.get_variable('B2',shape=(128),initializer = tf.keras.initializers.GlorotNormal()),
    'bd1':tf.compat.v1.get_variable('B3',shape=(128),initializer= tf.keras.initializers.GlorotNormal()),
    'out':tf.compat.v1.get_variable('B4',shape=(n_classes),initializer = tf.keras.initializers.GlorotNormal()),

}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
def conv_net(x,weight,biases):
    conv1 = conv2d(x,weight['wc1'],biases['bc1'])
    #max pooling (downsampling) this chooses the max value from 2*2 matrix 
    #and output 14*14 matrix
    conv1 = maxpool2d(conv1,k=2)
    
    conv2 = conv2d(conv1,weight['wc2'],biases['bc2'])
    #max pooling (downsampling) this chooses the max value from 2*2 matrix 
    #and output 14*14 matrix
    conv2 = maxpool2d(conv2,k=2)
    
    conv3 = conv2d(conv2,weight['wc3'],biases['bc3'])
    #max pooling (downsampling) this chooses the max value from 2*2 matrix 
    #and output 14*14 matrix
    conv3 = maxpool2d(conv3,k=2)
    
    # Fully connected layer
    #reshape conv2d output  to fit fully conncted layer input
    
    fc1 = tf.reshape(conv3,[-1,weight['wd1'].get_shape().as_list()[0]])
    fc1= tf.add(tf.matmul(fc1,weight['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    #output class prediction 
    #we multiply fully connectd layer with wieghts and add biases
    
    out = tf.add(tf.matmul(fc1,weight['out']),biases['out'])
    
    return out
        

In [13]:
pred = conv_net(x,weights,biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [14]:
#Here we check if index of max of predictd image is equal to actual label

correct_prediction = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))

#calculate accuracy across all image and avg them out

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [15]:
init = tf.compat.v1.global_variables_initializer()

In [17]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    for i in range(traing_iters):
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = data.train.labels[batch*batch_size:min((batch+1)*batch_size,len(data.train.labels))]
            
            # Run optimizer op
            # calculate batch loss and acc
            
            opt = sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
            
            loss,acc = sess.run([cost,accuracy],feed_dict={x:batch_x,y:batch_y})
            
            print("Iter"+str(i)+" , Loss = {} Training Accuracy {}".format(loss,acc))
            
            #Calculate accuracy for all 10000 mnist test images
            test_acc,valid_loss = sess.run([accuracy,cost],feed_dict={x:test_X,y:data.test.labels})
            train_loss.append(loss)
            test_loss.append(valid_loss)
            train_accuracy.append(acc)
            test_accuracy.append(test_acc)
            print("Testing Accuracy: {}".format(test_acc))
        
    
        

Iter0 , Loss = 2.2710533142089844 Training Accuracy 0.140625
Testing Accuracy: 0.0982000008225441
Iter0 , Loss = 2.2782835960388184 Training Accuracy 0.21875
Testing Accuracy: 0.20909999310970306
Iter0 , Loss = 2.2469449043273926 Training Accuracy 0.1484375
Testing Accuracy: 0.09799999743700027
Iter0 , Loss = 2.2399823665618896 Training Accuracy 0.109375
Testing Accuracy: 0.09799999743700027
Iter0 , Loss = 2.2233707904815674 Training Accuracy 0.109375
Testing Accuracy: 0.1062999963760376
Iter0 , Loss = 2.202786922454834 Training Accuracy 0.2421875
Testing Accuracy: 0.2184000015258789
Iter0 , Loss = 2.195564031600952 Training Accuracy 0.2890625
Testing Accuracy: 0.28049999475479126
Iter0 , Loss = 2.149296522140503 Training Accuracy 0.390625
Testing Accuracy: 0.3781999945640564
Iter0 , Loss = 2.0698728561401367 Training Accuracy 0.5234375
Testing Accuracy: 0.44510000944137573
Iter0 , Loss = 2.094212293624878 Training Accuracy 0.4140625
Testing Accuracy: 0.454800009727478
Iter0 , Loss = 2

Testing Accuracy: 0.9253000020980835
Iter0 , Loss = 0.33564725518226624 Training Accuracy 0.890625
Testing Accuracy: 0.9261999726295471
Iter0 , Loss = 0.38546013832092285 Training Accuracy 0.8828125
Testing Accuracy: 0.926800012588501
Iter0 , Loss = 0.3159756064414978 Training Accuracy 0.8828125
Testing Accuracy: 0.9289000034332275
Iter0 , Loss = 0.18609338998794556 Training Accuracy 0.9453125
Testing Accuracy: 0.9236999750137329
Iter0 , Loss = 0.17991264164447784 Training Accuracy 0.953125
Testing Accuracy: 0.9229999780654907
Iter0 , Loss = 0.20368246734142303 Training Accuracy 0.9375
Testing Accuracy: 0.9197999835014343
Iter0 , Loss = 0.24467350542545319 Training Accuracy 0.921875
Testing Accuracy: 0.9190999865531921
Iter0 , Loss = 0.26675140857696533 Training Accuracy 0.90625
Testing Accuracy: 0.9259999990463257
Iter0 , Loss = 0.22439099848270416 Training Accuracy 0.9453125
Testing Accuracy: 0.9297999739646912
Iter0 , Loss = 0.2458743304014206 Training Accuracy 0.9296875
Testing Acc

Testing Accuracy: 0.9526000022888184
Iter0 , Loss = 0.20639285445213318 Training Accuracy 0.9453125
Testing Accuracy: 0.9553999900817871
Iter0 , Loss = 0.2232537865638733 Training Accuracy 0.953125
Testing Accuracy: 0.9564999938011169
Iter0 , Loss = 0.2767963409423828 Training Accuracy 0.8828125
Testing Accuracy: 0.9591000080108643
Iter0 , Loss = 0.21195703744888306 Training Accuracy 0.9453125
Testing Accuracy: 0.9592000246047974
Iter0 , Loss = 0.08153131604194641 Training Accuracy 0.9765625
Testing Accuracy: 0.9552000164985657
Iter0 , Loss = 0.1129581406712532 Training Accuracy 0.96875
Testing Accuracy: 0.9539999961853027
Iter0 , Loss = 0.21481800079345703 Training Accuracy 0.9375
Testing Accuracy: 0.949400007724762
Iter0 , Loss = 0.08040788769721985 Training Accuracy 0.9765625
Testing Accuracy: 0.9473999738693237
Iter0 , Loss = 0.16960379481315613 Training Accuracy 0.953125
Testing Accuracy: 0.9496999979019165
Iter0 , Loss = 0.12411223351955414 Training Accuracy 0.9375
Testing Accura

KeyboardInterrupt: 